# Sıra Desen Madenciliği (Sequence Pattern Mining)

## Giriş

Sıra Desen Madenciliği, veri madenciliği ve makine öğrenmesi alanlarında önemli bir yere sahiptir. Bu algoritma, sıralı verilerde tekrar eden desenleri bulmaya çalışır. Bu tür veriler zaman serileri, metin verileri veya herhangi bir sıralı olaylar olabilir.

## Uygulama Alanları

- Pazarlama ve Müşteri Analizi: Müşteri alışveriş alışkanlıklarının analizi
- Biyoinformatik: DNA dizilimlerinin analizi
- Web Kullanıcı Davranışı: Web sayfası ziyaret sıralamalarının analizi

## Algoritmalar

Sıra desen madenciliği algoritmalarından bazıları AprioriAll, PrefixSpan, SPADE ve GSP algoritmalarıdır. Bu notta bu algoritmalara dair örnek kodlar yer almaktadır.


## Örnek 1: PrefixSpan Algoritması
PrefixSpan (Prefix-projected Sequential pattern mining) sık kullanılan bir sıralı desen madenciliği algoritmasıdır.

In [1]:
import itertools
from collections import defaultdict

def prefixspan(sequence, min_support):
    def frequent_items(sequences, min_support):
        items = defaultdict(int)
        for seq in sequences:
            for item in set(itertools.chain(*seq)):
                items[item] += 1
        return {item: count for item, count in items.items() if count >= min_support}

    def project_database(sequences, prefix):
        projected_db = []
        for seq in sequences:
            temp_seq = []
            for itemset in seq:
                if not temp_seq and set(prefix).issubset(itemset):
                    temp_seq = seq[seq.index(itemset)+1:]
                    break
            if temp_seq:
                projected_db.append(temp_seq)
        return projected_db

    def recursive_prefixspan(prefix, sequences, min_support):
        freq_patterns = []
        freq_items = frequent_items(sequences, min_support)
        for item, count in freq_items.items():
            new_prefix = prefix + [item]
            freq_patterns.append((new_prefix, count))
            projected_db = project_database(sequences, new_prefix)
            freq_patterns.extend(recursive_prefixspan(new_prefix, projected_db, min_support))
        return freq_patterns

    return recursive_prefixspan([], sequence, min_support)

# Örnek veri seti
sequences = [
    [['A'], ['B', 'C'], ['A', 'C']],
    [['A'], ['C'], ['B', 'C']],
    [['A', 'B'], ['D']],
    [['B', 'C'], ['A']],
    [['A'], ['B', 'D']],
]

# Minimum destek eşiği
min_support = 2

# Algoritmayı çalıştırma
frequent_patterns = prefixspan(sequences, min_support)

# Sonuçları yazdırma
print("Sık Desenler:")
for pattern in frequent_patterns:
    print(pattern)

Sık Desenler:
(['A'], 5)
(['A', 'B'], 3)
(['A', 'C'], 2)
(['A', 'D'], 2)
(['C'], 3)
(['C', 'A'], 2)
(['C', 'C'], 2)
(['B'], 5)
(['B', 'A'], 2)
(['D'], 2)


## Örnek 2: SPADE Algoritması
SPADE (Sequential Pattern Discovery using Equivalence classes) algoritması bir başka sıralı desen madenciliği algoritmasıdır.

In [2]:
import itertools
from collections import defaultdict

def spade(sequences, min_support):
    def frequent_items(sequences, min_support):
        items = defaultdict(int)
        for seq in sequences:
            for item in set(itertools.chain(*seq)):
                items[item] += 1
        return {item: count for item, count in items.items() if count >= min_support}

    def project_database(sequences, prefix):
        projected_db = []
        for seq in sequences:
            temp_seq = []
            for itemset in seq:
                if not temp_seq and set(prefix).issubset(itemset):
                    temp_seq = seq[seq.index(itemset)+1:]
                    break
            if temp_seq:
                projected_db.append(temp_seq)
        return projected_db

    def recursive_spade(prefix, sequences, min_support):
        freq_patterns = []
        freq_items = frequent_items(sequences, min_support)
        for item, count in freq_items.items():
            new_prefix = prefix + [item]
            freq_patterns.append((new_prefix, count))
            projected_db = project_database(sequences, new_prefix)
            freq_patterns.extend(recursive_spade(new_prefix, projected_db, min_support))
        return freq_patterns

    return recursive_spade([], sequences, min_support)

# Örnek veri seti
sequences = [
    [['A'], ['B', 'C'], ['A', 'C']],
    [['A'], ['C'], ['B', 'C']],
    [['A', 'B'], ['D']],
    [['B', 'C'], ['A']],
    [['A'], ['B', 'D']],
]

# Minimum destek eşiği
min_support = 2

# Algoritmayı çalıştırma
frequent_patterns = spade(sequences, min_support)

# Sonuçları yazdırma
print("Sık Desenler:")
for pattern in frequent_patterns:
    print(pattern)

Sık Desenler:
(['A'], 5)
(['A', 'B'], 3)
(['A', 'C'], 2)
(['A', 'D'], 2)
(['C'], 3)
(['C', 'A'], 2)
(['C', 'C'], 2)
(['B'], 5)
(['B', 'A'], 2)
(['D'], 2)


## Örnek 3: GSP Algoritması
GSP (Generalized Sequential Pattern) algoritması, sıralı desenleri bulmak için kullanılan başka bir algoritmadır.

In [3]:
import itertools
from collections import defaultdict

def gsp(sequences, min_support):
    def generate_candidates(frequent_patterns, k):
        candidates = []
        for i in range(len(frequent_patterns)):
            for j in range(i + 1, len(frequent_patterns)):
                seq1, seq2 = frequent_patterns[i], frequent_patterns[j]
                if seq1[:-1] == seq2[:-1]:
                    candidate = seq1 + [seq2[-1]]
                    if len(candidate) == k:
                        candidates.append(candidate)
        return candidates

    def count_support(sequences, candidates):
        candidate_counts = defaultdict(int)
        for seq in sequences:
            for candidate in candidates:
                if is_subsequence(candidate, seq):
                    candidate_counts[tuple(candidate)] += 1
        return candidate_counts

    def is_subsequence(subseq, seq):
        it = iter(seq)
        return all(any(set(item).issubset(set(s)) for s in it) for item in subseq)

    def filter_candidates(candidate_counts, min_support):
        return [list(candidate) for candidate, count in candidate_counts.items() if count >= min_support]

    # Başlangıçta 1-öğe sıralı desenlerini bulma
    freq_patterns = [[item] for item, count in frequent_items(sequences).items() if count >= min_support]
    all_freq_patterns = []

    k = 2
    while freq_patterns:
        all_freq_patterns.extend(freq_patterns)
        candidates = generate_candidates(freq_patterns, k)
        candidate_counts = count_support(sequences, candidates)
        freq_patterns = filter_candidates(candidate_counts, min_support)
        k += 1

    return all_freq_patterns

def frequent_items(sequences):
    item_counts = defaultdict(int)
    for seq in sequences:
        for item in set(itertools.chain(*seq)):
            item_counts[item] += 1
    return item_counts

# Örnek veri seti
sequences = [
    [['A'], ['B', 'C'], ['A', 'C']],
    [['A'], ['C'], ['B', 'C']],
    [['A', 'B'], ['D']],
    [['B', 'C'], ['A']],
    [['A'], ['B', 'D']],
]

# Minimum destek eşiği
min_support = 2

# Algoritmayı çalıştırma
frequent_patterns = gsp(sequences, min_support)

# Sonuçları yazdırma
print("Sık Desenler:")
for pattern in frequent_patterns:
    print(pattern)

Sık Desenler:
['A']
['C']
['B']
['D']
['A', 'C']
['A', 'B']
['A', 'D']
